In [ ]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import pylab as pl
%matplotlib inline

In [ ]:
trainingData = pd.read_csv("train.csv")

X_train = trainingData.drop("SalePrice", axis = 1)
X_train
Y_train = trainingData.SalePrice

def encodeEntireData(X_train):
    cache = []
    for feature in X_train.columns.values:
        label_encoder =LabelEncoder()
        X_train[feature]= label_encoder.fit_transform(X_train[feature])
        cache.append(label_encoder)

    return cache

encodeEntireData(X_train)

In [ ]:
trainingData.columns.values

In [ ]:
trainingData['YearsSinceRemodeled']  = trainingData['YrSold'] - trainingData['YearRemodAdd']
trainingData['YearsOld']  = trainingData['YrSold'] - trainingData['YearBuilt']
trainingData['YearsBeforeRemodel']  = trainingData['YearRemodAdd'] - trainingData['YearBuilt']
trainingData['YearsBeforeGarageBuild']  = trainingData['GarageYrBlt'] - trainingData['YearBuilt']
trainingData['TotalSF']  = trainingData['TotalBsmtSF'] + trainingData['1stFlrSF'] + trainingData['2ndFlrSF'] + trainingData['1stFlrSF']
trainingData['UnfinishedSF']  = (trainingData['LowQualFinSF'] + trainingData['BsmtUnfSF']) / trainingData['TotalSF']
trainingData['UnfinishedSFBasement']  = (trainingData['BsmtUnfSF']) / trainingData['TotalBsmtSF']
trainingData['FinishedSFBasement']  = 1 - (trainingData['BsmtUnfSF']) / trainingData['TotalBsmtSF']
trainingData['TotalOutsideSF']  = trainingData['WoodDeckSF'] + trainingData['OpenPorchSF'] + trainingData['EnclosedPorch'] + trainingData['3SsnPorch']+ trainingData['ScreenPorch'] + trainingData['PoolArea']
trainingData['PropertySF']  = trainingData['TotalSF'] + trainingData['TotalOutsideSF'] + trainingData['MasVnrArea'] + trainingData['LotFrontage']
trainingData['grassArea'] = trainingData['PropertySF']  - trainingData['LotArea'] 
trainingData['bathrooms'] = trainingData['BsmtFullBath']  + trainingData['BsmtHalfBath'] +trainingData['FullBath']  + trainingData['HalfBath'] 
trainingData['numberBedAndBath'] = trainingData['BedroomAbvGr']  + trainingData['FullBath'] 


In [ ]:
corrMatrix = trainingData.corr(method='pearson')

In [ ]:
corrMatrix['SalePrice']

In [ ]:
pd.plotting.scatter_matrix(trainingData[['FinishedSFBasement', 'TotalSF', 'UnfinishedSFBasement','SalePrice']])

In [ ]:
pd.plotting.scatter_matrix(trainingData[['YearsSinceRemodeled', 'YearsOld', 'YearsBeforeRemodel','SalePrice']])

In [ ]:
X_train = trainingData.drop("SalePrice", axis = 1)
encodeEntireData(X_train)

corrMatrix2 = X_train.corr(method='pearson')

In [ ]:
pd.set_option('display.max_rows', None)
corrMatrix2['SalePrice']

In [ ]:
from xgboost import XGBRegressor
X_train2, X_test, Y_train2, Y_test = train_test_split(X_train, Y_train, test_size=0.33, random_state=42)

model=XGBRegressor(n_estimators = 1000)
model.fit(X_train2, Y_train2, eval_set=[(X_test, Y_test)], early_stopping_rounds = 10)

In [ ]:
Y_pred = model.predict(X_test)
np.sum(abs(np.divide(Y_test - Y_pred, Y_test)))/len(Y_test)

In [ ]:
clf2 = RandomForestRegressor(n_estimators = 1000)
clf2.fit(X_train2, Y_train2)

print(clf2.score(X_train2,Y_train2))
print(clf2.score(X_test,Y_test))
y = clf2.feature_importances_
x = np.linspace(0, len(y)-1, len(y))
fig=plt.figure()
plt.plot(x,y)

In [ ]:
Y_pred = clf2.predict(X_test)
np.sum(abs(np.divide(Y_test - Y_pred, Y_test)))/len(Y_test)

In [ ]:
data = np.array([X_train.columns.values, y]).T
sortedArr = data[data[:,1].argsort()]

In [ ]:
sortedArr[0:16,0]